In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt

In [3]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

In [4]:
train_dataset=datasets.CIFAR10(root="./data",train=True,transform=transform,download=True)
test_dataset=datasets.CIFAR10(root="./data",train=False,transform=transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_subset=Subset(train_dataset,range(400))
test_subset=Subset(test_dataset,range(100))

In [6]:
train_loader=DataLoader(train_subset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_subset,batch_size=64,shuffle=False)

In [7]:
from torch import max_pool2d, relu


class CNNWithBNDropout(nn.Module):

    def __init__(self):
        super(CNNWithBNDropout,self).__init__()
        self.conv_block1=nn.Sequential(
            nn.Conv2d(3,32,kernel_size=3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.conv_block2=nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.dense1=nn.Linear(64*8*8,512)
        self.dense2=nn.Linear(512,10)

        self.droput=nn.Dropout(0.5)
        self.relu=nn.ReLU()
        self.flatten=nn.Flatten()


    def forward(self,x):

        x=self.conv_block1(x)
        x=self.conv_block2(x)
        x=self.flatten(x)
        x=self.dense1(x)
        x=self.relu(x)
        x=self.droput(x)
        x=self.dense2(x)
        

        return x

model=CNNWithBNDropout()
optimizer=optim.Adam(model.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()



In [8]:
def train_loop(epochs):

    for epoch in range(epochs):
        
        model.train()
        train_loss=0
        correct_train=0
        total_train=0

        for data,target in train_loader:

            output=model(data)
            loss=criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss+=loss.item()
            predicted=torch.argmax(output.data,dim=1)
            total_train+=output.size(0)
            correct_train+=(predicted==target).sum().item()

        avg_train_loss=train_loss/len(train_loader)
        train_acc= 100 * correct_train/total_train

        model.eval()

        test_loss=0
        correct_test=0
        total_test=0
        with torch.no_grad():
            for data,target in test_loader:

                output=model(data)
                loss=criterion(output,target)
                test_loss+=loss.item()
                predicted=torch.argmax(output.data,dim=1)
                total_test+=output.size(0)
                correct_test+=(predicted==target).sum().item()

            avg_test_loss=train_loss/len(test_loader)
            test_acc= 100 * correct_test/total_test  
            print(f'Epoch:{epoch+1}, train loss:{avg_train_loss}, test_loss:{avg_test_loss}, train accuracy:{train_acc}, test accuracy:{test_acc}')      

train_loop(30)


Epoch:1, train loss:3.4386889934539795, test_loss:12.035411477088928, train accuracy:13.5, test accuracy:22.0
Epoch:2, train loss:2.3843421254839217, test_loss:8.345197439193726, train accuracy:23.0, test accuracy:19.0
Epoch:3, train loss:2.0036573580333163, test_loss:7.012800753116608, train accuracy:26.75, test accuracy:30.0
Epoch:4, train loss:1.7995039054325648, test_loss:6.298263669013977, train accuracy:33.0, test accuracy:37.0
Epoch:5, train loss:1.7297826324190413, test_loss:6.054239213466644, train accuracy:40.25, test accuracy:39.0
Epoch:6, train loss:1.5419172729764665, test_loss:5.396710455417633, train accuracy:46.0, test accuracy:30.0
Epoch:7, train loss:1.5412216016224451, test_loss:5.394275605678558, train accuracy:45.25, test accuracy:42.0
Epoch:8, train loss:1.508212379046849, test_loss:5.278743326663971, train accuracy:46.5, test accuracy:32.0
Epoch:9, train loss:1.3754359313419886, test_loss:4.8140257596969604, train accuracy:51.0, test accuracy:31.0
Epoch:10, train

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

# Data preprocessing and loading with augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class CNNWithBNDropout(nn.Module):
    def __init__(self):
        super(CNNWithBNDropout, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.dense1 = nn.Linear(64 * 8 * 8, 512)
        self.dense2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.dense2(x)
        return x

model = CNNWithBNDropout()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

def train_loop(epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for data, target in train_loader:
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predicted = torch.argmax(output.data, dim=1)
            total_train += target.size(0)
            correct_train += (predicted == target).sum().item()

        avg_train_loss = train_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train

        model.eval()
        test_loss = 0
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for data, target in test_loader:
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item()
                predicted = torch.argmax(output.data, dim=1)
                total_test += target.size(0)
                correct_test += (predicted == target).sum().item()

        avg_test_loss = test_loss / len(test_loader)
        test_acc = 100 * correct_test / total_test

        print(f'Epoch: [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_acc:.4f}%, Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}%')

train_loop(30)


Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 